In [57]:
import os, sys, pickle
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
from math import ceil
from random import randint
import matplotlib.pyplot as plt
from shared.autoencoderHelpers import read_n_images, generate_img_from_folder, get_input_shape, get_num_examples, plot_history, get_images, bgr2rgb, plot_reconstruction
from shared.classifierHelpers import generate_img_label_from_folder
DATA_DIR = '../data'
import csv

In [77]:
def loadModel(configName, method='untrained'):
    path = '../models/classifiers/'+configName+'/'+method+'/'
    modelPath = path + 'model'
    historyPath = path +'history.pickle'
    
    model = tf.keras.models.load_model(modelPath)
    with open(historyPath, 'rb') as f:
        history = pickle.load(f)
        
    return (model,history)

def loadLabels(method,dataPath):
    path = "{}labels.pickle".format(dataPath)
    with open(path,'rb') as f:
        labels = pickle.load(f)
    labels = labels[method]
    labelsOnly = [item[1] for item in labels]
    return labelsOnly

def predictions(method,dataPath,model):
    BATCH_SIZE_TEST = 32
    NUM_SAMPLES_TEST = get_num_examples(dataPath, method)
    STEPS_TEST = ceil(NUM_SAMPLES_TEST/BATCH_SIZE_TEST)
    results =model.predict_generator(generate_img_label_from_folder(dataPath, method, BATCH_SIZE_TEST),steps=STEPS_TEST)
    results = results.tolist()
    return results

def accuracy(labels,predictions):
    
    def harden(predictionPair):
        if predictionPair[0] > predictionPair[1]:
            return [1,0]
        else:
            return [0,1]
    
    hardPredictions = [harden(prediction) for prediction in predictions]
    correctClassifications = 0.0
    
    for i in range(len(labels)):
        label = labels[i]
        prediction = hardPredictions[i]
        if (label[0] == 1 and prediction[0] == 1) or (label[1] == 1 and prediction[1] == 1):
            correctClassifications +=1.0
            
    accuracy = correctClassifications/len(labels)
    return accuracy

def saveCSV(actual,predicted,filename):
    
    data = [[actual[i][0],actual[i][1],predicted[i][0],predicted[i][1]] for i in range(len(actual))]
    
    with open(filename,'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    f.close()
    
    return True

def process(model,modelName):
    pred = predictions('test','../data/',model)
    labs = loadLabels('test','../data/')
    saveCSV(labs,pred,'../temp/test_{}.csv'.format(modelName))

    pred = predictions('validation','../data/',model)
    labs = loadLabels('validation','../data/')
    saveCSV(labs,pred,'../temp/validation_{}.csv'.format(modelName))

    pred = predictions('training','../data/',model)
    labs = loadLabels('training','../data/')
    saveCSV(labs,pred,'../temp/training_{}.csv'.format(modelName))
    

In [4]:
(model1_frozen,history) = loadModel('config1','frozen')
(model1_melted,history) = loadModel('config1','melted')
(model1_untrained,history) = loadModel('config1','untrained')

(model2_frozen,history) = loadModel('config2','frozen')
(model2_melted,history) = loadModel('config2','melted')
(model2_untrained,history) = loadModel('config2','untrained')

(model3_frozen,history) = loadModel('config3','frozen')
(model3_melted,history) = loadModel('config3','melted')
(model3_untrained,history) = loadModel('config3','untrained')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [56]:
preds = predictions('test','../data/',model3_untrained)
labs = loadLabels('test','../data/')

0.8728323699421965

In [ ]:
process(model1_frozen,'config1_frozen')
process(model1_melted,'config1_melted')
process(model1_untrained,'config1_untrained')
process(model2_frozen,'config2_frozen')
process(model2_melted,'config2_melted')
process(model2_untrained,'config2_untrained')
process(model3_frozen,'config3_frozen')
process(model3_melted,'config3_melted')
process(model3_untrained,'config3_untrained')

In [75]:
len(labs)

520